### weather大类有clear, partly_cloudy, haze, cloudy四种。本代码将训练集中含有这四个标签的图片分类了出来。
fname，fname1，fname2，fname3是最终结果。这四个是数组类型，里面含有clear, partly_cloudy, haze, cloudy对应图像在训练集中的名字

In [44]:
import sys
import os
import subprocess

from six import string_types

# Make sure you have all of these packages installed, e.g. via pip
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
import cv2
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import regularizers
%matplotlib inline

Using TensorFlow backend.


In [2]:
PLANET_KAGGLE_ROOT = os.path.abspath("/Users/sunchenxi/Desktop/lab/kaggle/amazon/")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

In [3]:
#读取train set标签
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
# 创建一个list里面有所有标签
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [5]:
label_list

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [6]:
#进行one－hot编码 数据预处理
#对每一个图像的标签进行标记，有此标签为1
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
labels_df.head()#显示几个看看

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [7]:
#统计每一个单标签的图像个数
labels_df_sum=labels_df[label_list].sum().sort_values()
labels_df_sum

blow_down               98
conventional_mine      100
slash_burn             209
blooming               332
artisinal_mine         339
selective_logging      340
bare_ground            862
cloudy                2089
haze                  2697
habitation            3660
cultivation           4477
partly_cloudy         7261
water                 7411
road                  8071
agriculture          12315
clear                28431
primary              37513
dtype: int64

In [8]:
#获取含有tags标签的n张图片
def sample_images(tags, n=None):
    """Randomly sample n images with the specified tags."""
    condition = True
    if isinstance(tags, string_types): #tag是否是string类型
        raise ValueError("Pass a list of tags, not a single tag.")
    for tag in tags:
        condition = condition & labels_df[tag] == 1
    if n is not None:
        return labels_df[condition].sample(n)
    else:
        return labels_df[condition]

In [9]:
def load_image(filename):
    '''Look through the directory tree to find the image you specified
    (e.g. train_10.tif vs. train_10.jpg)'''
    for dirname in os.listdir(PLANET_KAGGLE_ROOT):
        path = os.path.abspath(os.path.join(PLANET_KAGGLE_ROOT, dirname, filename))
        if os.path.exists(path):
            print('Found image {}'.format(path))
            return io.imread(path)
    print('Load failed: could not find image {}'.format(path))
 
 #将图片在数组中的位置变成名称
def sample_to_fname(sample_df, row_idx, suffix='jpg'):
    '''Given a dataframe of sampled images, get the
    corresponding filename.'''
    fname = sample_df.get_value(sample_df.index[row_idx], 'image_name')
    return '{}.{}'.format(fname, suffix)

In [10]:
def plot_rgbn_histo(r, g, b):
    for slice_, name, color in ((r,'r', 'red'),(g,'g', 'green'),(b,'b', 'blue')):
        plt.hist(slice_.ravel(), bins=100, 
                 range=[0,rgb_image.max()], 
                 label=name, color=color, histtype='step')
    plt.legend()

##### 计算出label＝clear的图像个数

In [11]:
len(labels_df)

40479

In [12]:
n=0
for i in range(len(labels_df)):
    if(labels_df["clear"][i]==1):
        n=n+1

In [13]:
n

28431

#### 获得含有clear的图片的名字

In [14]:
fname=[0]*n
s = sample_images(['clear'], n=n)
for i in range(n):
    fname[i]= sample_to_fname(s, i)#获得了文件名

##### 图片名字是：

In [15]:
fname

['train_11944.jpg',
 'train_27891.jpg',
 'train_40329.jpg',
 'train_2442.jpg',
 'train_38591.jpg',
 'train_2223.jpg',
 'train_31127.jpg',
 'train_26803.jpg',
 'train_12289.jpg',
 'train_15623.jpg',
 'train_34844.jpg',
 'train_1525.jpg',
 'train_5768.jpg',
 'train_27650.jpg',
 'train_20452.jpg',
 'train_22367.jpg',
 'train_34219.jpg',
 'train_973.jpg',
 'train_33411.jpg',
 'train_15146.jpg',
 'train_5016.jpg',
 'train_31643.jpg',
 'train_24416.jpg',
 'train_18651.jpg',
 'train_29804.jpg',
 'train_14713.jpg',
 'train_20666.jpg',
 'train_25501.jpg',
 'train_7371.jpg',
 'train_670.jpg',
 'train_38622.jpg',
 'train_4486.jpg',
 'train_17364.jpg',
 'train_32413.jpg',
 'train_27974.jpg',
 'train_15403.jpg',
 'train_16594.jpg',
 'train_37316.jpg',
 'train_11836.jpg',
 'train_34294.jpg',
 'train_10771.jpg',
 'train_13274.jpg',
 'train_35221.jpg',
 'train_23322.jpg',
 'train_28083.jpg',
 'train_13287.jpg',
 'train_21352.jpg',
 'train_38374.jpg',
 'train_2794.jpg',
 'train_12175.jpg',
 'train_3134

##### 同样的方法获取含有partly_cloudy标签的图片名称

In [16]:
n1=0
for i in range(len(labels_df)):
    if(labels_df["partly_cloudy"][i]==1):
        n1=n1+1

In [17]:
n1

7261

In [18]:
fname1=[0]*n1
s = sample_images(['partly_cloudy'], n=n1)
for i in range(n1):
    fname1[i]= sample_to_fname(s, i)#获得了文件名

In [19]:
fname1

['train_4010.jpg',
 'train_33357.jpg',
 'train_19614.jpg',
 'train_385.jpg',
 'train_39228.jpg',
 'train_32638.jpg',
 'train_34268.jpg',
 'train_21456.jpg',
 'train_8991.jpg',
 'train_22556.jpg',
 'train_26391.jpg',
 'train_36863.jpg',
 'train_9453.jpg',
 'train_2222.jpg',
 'train_19264.jpg',
 'train_7868.jpg',
 'train_34559.jpg',
 'train_20413.jpg',
 'train_22943.jpg',
 'train_7912.jpg',
 'train_24383.jpg',
 'train_32006.jpg',
 'train_29557.jpg',
 'train_30150.jpg',
 'train_15255.jpg',
 'train_7734.jpg',
 'train_9849.jpg',
 'train_8310.jpg',
 'train_37456.jpg',
 'train_1620.jpg',
 'train_6359.jpg',
 'train_24384.jpg',
 'train_36178.jpg',
 'train_37460.jpg',
 'train_33358.jpg',
 'train_2284.jpg',
 'train_975.jpg',
 'train_9013.jpg',
 'train_29976.jpg',
 'train_19352.jpg',
 'train_2492.jpg',
 'train_14756.jpg',
 'train_22657.jpg',
 'train_8005.jpg',
 'train_22858.jpg',
 'train_11258.jpg',
 'train_34764.jpg',
 'train_10530.jpg',
 'train_35857.jpg',
 'train_5681.jpg',
 'train_27667.jpg',


##### 获取含有haze的图片的名称

In [20]:
n2=0
for i in range(len(labels_df)):
    if(labels_df["haze"][i]==1):
        n2=n2+1
fname2=[0]*n1
s = sample_images(['haze'], n=n2)
for i in range(n2):
    fname2[i]= sample_to_fname(s, i)#获得了文件名        

In [21]:
fname

['train_11944.jpg',
 'train_27891.jpg',
 'train_40329.jpg',
 'train_2442.jpg',
 'train_38591.jpg',
 'train_2223.jpg',
 'train_31127.jpg',
 'train_26803.jpg',
 'train_12289.jpg',
 'train_15623.jpg',
 'train_34844.jpg',
 'train_1525.jpg',
 'train_5768.jpg',
 'train_27650.jpg',
 'train_20452.jpg',
 'train_22367.jpg',
 'train_34219.jpg',
 'train_973.jpg',
 'train_33411.jpg',
 'train_15146.jpg',
 'train_5016.jpg',
 'train_31643.jpg',
 'train_24416.jpg',
 'train_18651.jpg',
 'train_29804.jpg',
 'train_14713.jpg',
 'train_20666.jpg',
 'train_25501.jpg',
 'train_7371.jpg',
 'train_670.jpg',
 'train_38622.jpg',
 'train_4486.jpg',
 'train_17364.jpg',
 'train_32413.jpg',
 'train_27974.jpg',
 'train_15403.jpg',
 'train_16594.jpg',
 'train_37316.jpg',
 'train_11836.jpg',
 'train_34294.jpg',
 'train_10771.jpg',
 'train_13274.jpg',
 'train_35221.jpg',
 'train_23322.jpg',
 'train_28083.jpg',
 'train_13287.jpg',
 'train_21352.jpg',
 'train_38374.jpg',
 'train_2794.jpg',
 'train_12175.jpg',
 'train_3134

##### 获取含有cloudy标签的图像名称

In [22]:
n3=0
for i in range(len(labels_df)):
    if(labels_df["cloudy"][i]==1):
        n3=n3+1
fname3=[0]*n1
s = sample_images(['cloudy'], n=n3)
for i in range(n3):
    fname3[i]= sample_to_fname(s, i)#获得了文件名    

In [23]:
fname3

['train_28923.jpg',
 'train_6262.jpg',
 'train_4038.jpg',
 'train_35027.jpg',
 'train_9770.jpg',
 'train_7419.jpg',
 'train_21210.jpg',
 'train_4263.jpg',
 'train_26320.jpg',
 'train_24472.jpg',
 'train_29353.jpg',
 'train_1520.jpg',
 'train_33510.jpg',
 'train_33291.jpg',
 'train_343.jpg',
 'train_717.jpg',
 'train_21336.jpg',
 'train_35495.jpg',
 'train_1256.jpg',
 'train_13248.jpg',
 'train_31588.jpg',
 'train_22405.jpg',
 'train_24649.jpg',
 'train_6093.jpg',
 'train_18281.jpg',
 'train_38446.jpg',
 'train_7759.jpg',
 'train_1360.jpg',
 'train_3297.jpg',
 'train_18175.jpg',
 'train_9062.jpg',
 'train_18386.jpg',
 'train_31887.jpg',
 'train_7192.jpg',
 'train_20250.jpg',
 'train_30255.jpg',
 'train_21471.jpg',
 'train_39163.jpg',
 'train_19396.jpg',
 'train_17552.jpg',
 'train_3837.jpg',
 'train_5391.jpg',
 'train_7859.jpg',
 'train_15622.jpg',
 'train_36964.jpg',
 'train_11118.jpg',
 'train_3096.jpg',
 'train_30487.jpg',
 'train_30514.jpg',
 'train_2164.jpg',
 'train_25647.jpg',
 '

#### 获取图片分类放入新文件夹---------

In [24]:
clear_img = [mpimg.imread(PLANET_KAGGLE_ROOT+'/train-jpg/'+ img) for img in fname[:100]]

In [25]:
partly_cloudy_img=[mpimg.imread(PLANET_KAGGLE_ROOT+'/train-jpg/'+ img) for img in fname1[:100]]

In [27]:
haze_cloudy_img=[mpimg.imread(PLANET_KAGGLE_ROOT+'/train-jpg/'+ img) for img in fname2[:100]]

In [28]:
cloudy_img=[mpimg.imread(PLANET_KAGGLE_ROOT+'/train-jpg/'+ img) for img in fname3[:100]]

In [29]:
weather_img=clear_img+partly_cloudy_img+haze_cloudy_img+cloudy_img

In [30]:
len(weather_img)

400

In [57]:
clear=['1']*100

In [58]:
partly=['2']*100

In [59]:
haze=['3']*100

In [66]:
cloudy=['4']*100

In [67]:
weater_labels=clear+partly+haze+cloudy

In [68]:
weater_labels

['1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2'

In [69]:
resized_images = [np.reshape(cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA), [1,64,64,4]) for img in weather_img]

In [70]:
train_size = int(len(resized_images)*0.9)
train_x = np.vstack(resized_images[:train_size])
test_x = resized_images[train_size:]
train_y = np.vstack(weater_labels[:train_size])
test_y = weater_labels[train_size:]

In [71]:
optimizer = RMSprop(lr=1e-3)
objective = 'binary_crossentropy'
def convnet_v1():
    
    model = Sequential()

    model.add(Convolution2D(32, 4, 4, border_mode='same', input_shape=(64, 64, 4), activation='relu'))
    model.add(Convolution2D(32, 4, 4, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Convolution2D(64, 4, 4, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    
    model.add(Convolution2D(128, 4, 4, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    

    model.add(Flatten())
    model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.3)))
    
    model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.3)))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    return model


model = convnet_v1()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (4, 4), padding="same", activation="relu", input_shape=(64, 64, 4...)`
  import sys
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (4, 4), padding="same", activation="relu")`
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), padding="same", activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), padding="same", activation="relu")`
  


In [72]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

history = LossHistory()
model.fit(train_x, train_y, batch_size=32, epochs=10,
          validation_split=0.25, verbose=1, shuffle=True, callbacks=[history])

Train on 270 samples, validate on 90 samples
Epoch 1/10
270/270 [==============================] - 9s - loss: 133.3179 - acc: 0.3704 - val_loss: 75.3338 - val_acc: 0.0000e+00
Epoch 2/10
270/270 [==============================] - 9s - loss: 89.8099 - acc: 0.3704 - val_loss: 44.1489 - val_acc: 0.0000e+00
Epoch 3/10
270/270 [==============================] - 9s - loss: 62.3440 - acc: 0.3704 - val_loss: 20.8821 - val_acc: 0.0000e+00
Epoch 4/10
270/270 [==============================] - 10s - loss: 41.2897 - acc: 0.3704 - val_loss: 2.6537 - val_acc: 0.0000e+00
Epoch 5/10
270/270 [==============================] - 9s - loss: 24.7991 - acc: 0.3704 - val_loss: -11.5221 - val_acc: 0.0000e+00
Epoch 6/10
270/270 [==============================] - 8s - loss: 12.1112 - acc: 0.3704 - val_loss: -22.2152 - val_acc: 0.0000e+00
Epoch 7/10
270/270 [==============================] - 9s - loss: 2.6958 - acc: 0.3704 - val_loss: -29.9326 - val_acc: 0.0000e+00
Epoch 8/10
270/270 [=============================

In [75]:
score = model.evaluate(train_x, train_y, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  

('Test score:', -19.297021092308892)
('Test accuracy:', 0.27777777777777779)
